In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
from ultralytics import YOLO
import cv2
import time
import numpy as np

In [2]:
lr= 0.000025341814210920752
epoch = 68
resize = 148
dropout= 0.02071552291233333


maxpooling= [2,5]

batch_size = 128
num_filters= [18,5]
kernel_size = [10,8]
resize = 148
   

In [3]:
transform = transforms.Compose([
    transforms.Resize((resize, resize)),  # Chọn kích thước ảnh sau khi augment
    transforms.ToTensor(),  # Chuyển ảnh sang tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Chuẩn hóa tensor
])

In [4]:
class BMIModel(nn.Module):
    def __init__(self, resize = 148, num_filters = [18,5], kernel_size =[10,8], maxpooling = [2,5]):
        super().__init__()
        self.resize = resize
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.maxpooling = maxpooling
        
        
        # input -> conv2d -> batchnorm -> maxpool2d ->conv2d -> batchnorm -> maxpool 
        
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)

        self.conv1 = nn.Conv2d(
                            in_channels= 3, 
                            out_channels=self.num_filters[0], 
                            kernel_size=self.kernel_size[0],
                            )

        self.conv2 = nn.Conv2d(
                            in_channels=self.num_filters[0], 
                            out_channels=self.num_filters[1], 
                            kernel_size=self.kernel_size[1],
                            )
        # self.conv3 = nn.Conv2d(
        #                     in_channels=self.num_filters[1], 
        #                     out_channels=1, 
        #                     kernel_size=3,
        #                     )
        # nn.init.xavier_uniform_(self.conv1.weight)
        # nn.init.zeros_(self.conv1.bias)
        # nn.init.xavier_uniform_(self.conv2.weight)
        # nn.init.zeros_(self.conv2.bias)

        self.f_activation1 = nn.ReLU()
        self.f_activation2 = nn.ReLU()
        
        


        # torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)
        self.batchNorm1 = nn.BatchNorm2d(num_features=self.num_filters[0])
        self.batchNorm2 = nn.BatchNorm2d(num_features=self.num_filters[1])

        # torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        
        self.maxpooling1 = nn.MaxPool2d(kernel_size=self.maxpooling[0])
        self.maxpooling2 = nn.MaxPool2d(kernel_size=self.maxpooling[1])
        
        self.h_conv1 = int((self.resize- self.kernel_size[0]) + 1)
        #self.w_conv1 = int((self.shape_data[2] - self.kernel_size[0]) + 1)

        self.h_maxpool1 = int((self.h_conv1 - self.maxpooling[0])/self.maxpooling[0] + 1)
        #self.w_maxpool1 = int((self.w_conv1 - self.maxpooling[0])/self.maxpooling[0] + 1)  

        self.h_conv2 = int((self.h_maxpool1 - self.kernel_size[1]) + 1)
        #self.w_conv2 = int((self.w_maxpool1 - self.kernel_size[1]) + 1)

        self.h_maxpool2= int((self.h_conv2 - self.maxpooling[1])/self.maxpooling[1] + 1)
        #self.w_maxpool2 = int((self.w_conv2 - self.maxpooling[1])/self.maxpooling[1] + 1)  

        # self.h_conv3 = int((self.h_maxpool2 - 3) + 1)
        # self.w_conv3 = int((self.w_maxpool2 - 3) + 1)
        
        
        
        self.linear1 = nn.Linear(self.h_maxpool2*self.h_maxpool2*num_filters[1], 120)
        self.f_activation3 = nn.ReLU()
        self.linear2 = nn.Linear(120, 84)
        self.f_activation4= nn.ReLU()
        self.linear3 = nn.Linear(84, 1)
        
        
        
        
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.f_activation1(x)
        x = self.batchNorm1(x)
        x = self.maxpooling1(x)
        
        
        x = self.conv2(x)
        x = self.f_activation2(x)
        x = self.batchNorm2(x)
        x = self.maxpooling2(x)
        x = torch.flatten(x, start_dim=1)
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        
        return x

In [6]:
model = BMIModel()
model.load_state_dict(torch.load('(10,8)kernel_(18,5)filter_(2,5)maxpool_2.5341814210920752e-05lr.pt',  map_location=torch.device('cpu'))) # it takes the loaded dictionary, not the path file itself
model.eval()
face_detection = YOLO('best.pt')

In [23]:
# Nhan dien BMI qua hinh anh khuon mat
with torch.no_grad():
    image = Image.open('m_021.jpg').convert("RGB")
   
    image = transform(image)
    image = image.unsqueeze_(0)
    y = model(image)

    print(y)

tensor([[23.8889]])


In [7]:
# Cat hinh anh khuon mat qua hinh anh toan than
img_path = "img_2541.bmp"

results = face_detection(img_path) 
boxes = results[0].boxes
img = cv2.imread(img_path)



image 1/1 C:\Users\DELL\AI\Project\BMI\img_2541.bmp: 640x576 1 FACE, 371.7ms
Speed: 10.8ms preprocess, 371.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 576)


In [8]:
# Nhan dien BMI qua hinh anh da cat
for box in boxes:
    top_left_x = int(box.xyxy.tolist()[0][0])
    top_left_y = int(box.xyxy.tolist()[0][1])
    bottom_right_x = int(box.xyxy.tolist()[0][2])
    bottom_right_y = int(box.xyxy.tolist()[0][3])
    croppedImg = img[top_left_y+1:bottom_right_y, top_left_x+1:bottom_right_x]
    cv2.rectangle(img, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (50, 200, 129), 2)
cv2.imwrite(f"Cropped.jpg", croppedImg)
with torch.no_grad():
    image = Image.open("Cropped.jpg").convert("RGB")
    image = transform(image)
    image = image.unsqueeze_(0)
    bmi_output = model(image)
    bmi_value = bmi_output.item()
    print(bmi_output)
    
for box in boxes:
    top_left_x = int(box.xyxy.tolist()[0][0])
    top_left_y = int(box.xyxy.tolist()[0][1])
    bottom_right_x = int(box.xyxy.tolist()[0][2])
    bottom_right_y = int(box.xyxy.tolist()[0][3])
    croppedImg = img[top_left_y+1:bottom_right_y, top_left_x+1:bottom_right_x]
    cv2.rectangle(img, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (50, 200, 129), 1)
    # Display BMI result on the original image
bmi_text = f'BMI: {bmi_value:.2f}'
cv2.putText(img, bmi_text, (top_left_x, top_left_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 255, 0), 1)
cv2.imshow('window', img)
cv2.imwrite(f"BMI.jpg", img)
cv2.waitKey(0) 
  
# closing all open windows 
cv2.destroyAllWindows() 



tensor([[22.7053]])
